In [78]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import joblib

In [13]:
# Python ≥3.5 is required
import pandas as pd
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.4 is required in this notebook
# Earlier 2.x versions will mostly work the same, but with a few bugs
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.4"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [14]:
df = pd.read_csv("/Users/karthiktalluri/Downloads/Deep_learning_signals_project/predicting_complex_numbers/merged.csv")

In [15]:
df.head()

,Unnamed: 0,Real Part of Input,Real Part of Output,Imag Part of Input,Imag Part of Output
0,0,0.280737,0.770811,-0.427208,-0.250749
1,1,0.385676,0.627422,-0.032750,0.244178
2,2,0.110913,-0.024803,0.321741,0.609490
3,3,-0.370705,-0.764049,0.281494,0.041161
4,4,-0.591695,-0.366071,-0.182893,-0.816991


In [16]:
# import data
data = pd.read_csv("merged.csv")
# Drop first column of dataframe
data = data.iloc[: , 1:]
df = data
#Rearranging columns of the dataset
df = df[['Real Part of Output', 'Imag Part of Output', 'Real Part of Input', 'Imag Part of Input']]

In [17]:
df.head()

,Real Part of Output,Imag Part of Output,Real Part of Input,Imag Part of Input
0,0.770811,-0.250749,0.280737,-0.427208
1,0.627422,0.244178,0.385676,-0.032750
2,-0.024803,0.609490,0.110913,0.321741
3,-0.764049,0.041161,-0.370705,0.281494
4,-0.366071,-0.816991,-0.591695,-0.182893


In [18]:
xtrain = df[["Real Part of Output", "Imag Part of Output"]]
labels = df[["Real Part of Input", "Imag Part of Input"]]

In [19]:
x, x_test, y, y_test = train_test_split(xtrain,labels,test_size=0.2,train_size=0.8, random_state= 42)
x_train, x_cv, y_train, y_cv = train_test_split(x,y,test_size = 0.25,train_size =0.75, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_valid = scaler.transform(x_cv)
X_test = scaler.transform(x_test)

In [20]:
x.head()

,Real Part of Output,Imag Part of Output
4063089,0.777741,-0.406226
1177619,0.148379,-0.988459
3179449,-0.427355,0.004795
695596,0.266688,-0.664563
1737797,-0.874405,0.046547


In [70]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape = x_train.shape[1:]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(2))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [71]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [64]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
keras_reg.fit(X_train, y_train, epochs=20,
              validation_data=(X_valid, y_cv),
              callbacks=[keras.callbacks.EarlyStopping(patience=2), lr_scheduler])

Epoch 1/20
76800/76800 [==============================] - 19s 244us/step - loss: 0.0106 - val_loss: 0.0062
Epoch 2/20
76800/76800 [==============================] - 19s 244us/step - loss: 0.0051 - val_loss: 0.0044
Epoch 3/20
76800/76800 [==============================] - 18s 237us/step - loss: 0.0039 - val_loss: 0.0036
Epoch 4/20
76800/76800 [==============================] - 18s 239us/step - loss: 0.0033 - val_loss: 0.0031
Epoch 5/20
76800/76800 [==============================] - 19s 247us/step - loss: 0.0029 - val_loss: 0.0028
Epoch 6/20
76800/76800 [==============================] - 19s 242us/step - loss: 0.0027 - val_loss: 0.0026
Epoch 7/20
76800/76800 [==============================] - 20s 254us/step - loss: 0.0025 - val_loss: 0.0025
Epoch 8/20
76800/76800 [==============================] - 19s 244us/step - loss: 0.0024 - val_loss: 0.0024
Epoch 9/20
76800/76800 [==============================] - 19s 244us/step - loss: 0.0023 - val_loss: 0.0023
Epoch 10/20
76800/76800 [============

In [65]:
param_distribs = {
    "n_hidden": [0, 1, 2, 3, 4, 5],
    "n_neurons": np.arange(1, 100)               .tolist(),
    "learning_rate": reciprocal(3e-4, 3e-2)      .rvs(1000).tolist(),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=20,
                  validation_data=(X_valid, y_cv),
                  callbacks=[keras.callbacks.EarlyStopping(patience=2), lr_scheduler])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/20
51200/51200 [==============================] - 23s 442us/step - loss: 0.0046 - val_loss: 0.0018
Epoch 2/20
51200/51200 [==============================] - 23s 444us/step - loss: 0.0015 - val_loss: 0.0014loss:  - ETA: 1s -
Epoch 3/20
51200/51200 [==============================] - 22s 430us/step - loss: 0.0013 - val_loss: 0.0013
Epoch 4/20
51200/51200 [==============================] - 22s 426us/step - loss: 0.0012 - val_loss: 0.0012
Epoch 5/20
51200/51200 [==============================] - 22s 423us/step - loss: 0.0012 - val_loss: 0.0012
Epoch 6/20
51200/51200 [==============================] - 22s 422us/step - loss: 0.0012 - val_loss: 0.0012
Epoch 7/20
51200/51200 [==============================] - 22s 428us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 8/20
51200/51200 [==============================] - 22s 424us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 9/20
51200/51200 [==============================] - 22s 423

51200/51200 [==============================] - 17s 337us/step - loss: 0.0033 - val_loss: 0.0032
Epoch 18/20
51200/51200 [==============================] - 17s 335us/step - loss: 0.0031 - val_loss: 0.0031
Epoch 19/20
51200/51200 [==============================] - 17s 338us/step - loss: 0.0030 - val_loss: 0.0030
Epoch 20/20
25600/25600 [==============================] - 5s 191us/step - loss: 0.0028
[CV] END learning_rate=0.0003457015717113411, n_hidden=5, n_neurons=16; total time= 5.9min
Epoch 1/20
51200/51200 [==============================] - 18s 345us/step - loss: 0.0383 - val_loss: 0.0192
Epoch 2/20
51200/51200 [==============================] - 17s 339us/step - loss: 0.0158 - val_loss: 0.0135
Epoch 3/20
51200/51200 [==============================] - 17s 339us/step - loss: 0.0122 - val_loss: 0.0113
Epoch 4/20
51200/51200 [==============================] - 17s 338us/step - loss: 0.0104 - val_loss: 0.0098
Epoch 5/20
51200/51200 [==============================] - 17s 340us/step - loss: 

51200/51200 [==============================] - 16s 319us/step - loss: 0.0017 - val_loss: 0.0016
Epoch 4/20
51200/51200 [==============================] - 16s 322us/step - loss: 0.0016 - val_loss: 0.0016
Epoch 5/20
51200/51200 [==============================] - 16s 319us/step - loss: 0.0015 - val_loss: 0.0015
Epoch 6/20
51200/51200 [==============================] - 16s 317us/step - loss: 0.0015 - val_loss: 0.0014
Epoch 7/20
51200/51200 [==============================] - 16s 320us/step - loss: 0.0014 - val_loss: 0.0014
Epoch 8/20
51200/51200 [==============================] - 16s 317us/step - loss: 0.0014 - val_loss: 0.0014
Epoch 9/20
51200/51200 [==============================] - 16s 317us/step - loss: 0.0014 - val_loss: 0.0014
Epoch 10/20
51200/51200 [==============================] - 16s 317us/step - loss: 0.0014 - val_loss: 0.0014
Epoch 11/20
51200/51200 [==============================] - 17s 324us/step - loss: 0.0014 - val_loss: 0.0014
Epoch 12/20
51200/51200 [=====================

51200/51200 [==============================] - 23s 444us/step - loss: 0.0011 - val_loss: 0.0012
Epoch 19/20
51200/51200 [==============================] - 23s 449us/step - loss: 0.0011 - val_loss: 0.0012
Epoch 20/20
25600/25600 [==============================] - 6s 239us/step - loss: 0.0011
[CV] END learning_rate=0.0018475234625589612, n_hidden=5, n_neurons=78; total time= 7.8min
Epoch 1/20
51200/51200 [==============================] - 23s 454us/step - loss: 0.0105 - val_loss: 0.0040
Epoch 2/20
51200/51200 [==============================] - 23s 448us/step - loss: 0.0029 - val_loss: 0.0024
Epoch 3/20
51200/51200 [==============================] - 23s 447us/step - loss: 0.0020 - val_loss: 0.0019
Epoch 4/20
51200/51200 [==============================] - 23s 457us/step - loss: 0.0017 - val_loss: 0.0016
Epoch 5/20
51200/51200 [==============================] - 23s 455us/step - loss: 0.0015 - val_loss: 0.0015
Epoch 6/20
51200/51200 [==============================] - 23s 450us/step - loss: 0

51200/51200 [==============================] - 14s 271us/step - loss: 0.0043 - val_loss: 0.0042
Epoch 8/20
51200/51200 [==============================] - 14s 267us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 9/20
51200/51200 [==============================] - 14s 269us/step - loss: 0.0037 - val_loss: 0.0037
Epoch 10/20
51200/51200 [==============================] - 14s 274us/step - loss: 0.0035 - val_loss: 0.0035
Epoch 11/20
51200/51200 [==============================] - 14s 265us/step - loss: 0.0033 - val_loss: 0.0033
Epoch 12/20
51200/51200 [==============================] - 14s 266us/step - loss: 0.0032 - val_loss: 0.0032
Epoch 13/20
51200/51200 [==============================] - 14s 266us/step - loss: 0.0031 - val_loss: 0.0030
Epoch 14/20
51200/51200 [==============================] - 14s 277us/step - loss: 0.0029 - val_loss: 0.0029
Epoch 15/20
51200/51200 [==============================] - 14s 266us/step - loss: 0.0028 - val_loss: 0.0029
Epoch 16/20
51200/51200 [=================

51200/51200 [==============================] - 17s 323us/step - loss: 0.0011 - val_loss: 0.0012
Epoch 10/20
51200/51200 [==============================] - 17s 324us/step - loss: 0.0011 - val_loss: 0.0012
Epoch 11/20
51200/51200 [==============================] - 16s 322us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 12/20
51200/51200 [==============================] - 17s 323us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 13/20
51200/51200 [==============================] - 16s 321us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 14/20
51200/51200 [==============================] - 16s 321us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 15/20
51200/51200 [==============================] - 17s 322us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 16/20
51200/51200 [==============================] - 16s 320us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 17/20
51200/51200 [==============================] - 16s 322us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 18/20
51200/51200 [===============

51200/51200 [==============================] - 14s 266us/step - loss: 0.0016 - val_loss: 0.0016
Epoch 10/20
51200/51200 [==============================] - 14s 270us/step - loss: 0.0016 - val_loss: 0.0016
Epoch 11/20
51200/51200 [==============================] - 14s 270us/step - loss: 0.0016 - val_loss: 0.0016
Epoch 12/20
51200/51200 [==============================] - 14s 268us/step - loss: 0.0015 - val_loss: 0.0015
Epoch 13/20
51200/51200 [==============================] - 14s 266us/step - loss: 0.0015 - val_loss: 0.0015
Epoch 14/20
51200/51200 [==============================] - 14s 268us/step - loss: 0.0015 - val_loss: 0.0015
Epoch 15/20
51200/51200 [==============================] - 19s 378us/step - loss: 0.0014 - val_loss: 0.0015
Epoch 16/20
51200/51200 [==============================] - 15s 286us/step - loss: 0.0014 - val_loss: 0.0014
Epoch 17/20
51200/51200 [==============================] - 14s 269us/step - loss: 0.0014 - val_loss: 0.0014
Epoch 18/20
51200/51200 [===============

51200/51200 [==============================] - 16s 317us/step - loss: 0.0012 - val_loss: 0.0012
Epoch 20/20
25600/25600 [==============================] - 5s 196us/step - loss: 0.0012
[CV] END learning_rate=0.005116171405379177, n_hidden=2, n_neurons=71; total time= 5.5min
Epoch 1/20
51200/51200 [==============================] - 16s 319us/step - loss: 0.0076 - val_loss: 0.0038
Epoch 2/20
51200/51200 [==============================] - 17s 325us/step - loss: 0.0029 - val_loss: 0.0025
Epoch 3/20
51200/51200 [==============================] - 17s 324us/step - loss: 0.0022 - val_loss: 0.0020
Epoch 4/20
51200/51200 [==============================] - 16s 318us/step - loss: 0.0018 - val_loss: 0.0018
Epoch 5/20
51200/51200 [==============================] - 16s 318us/step - loss: 0.0016 - val_loss: 0.0016
Epoch 6/20
51200/51200 [==============================] - 16s 321us/step - loss: 0.0015 - val_loss: 0.0015
Epoch 7/20
51200/51200 [==============================] - 16s 320us/step - loss: 0.0

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x2adc8dfd0>,
                   param_distributions={'learning_rate': [0.00875021488452691,
                                                          0.003747679319241924,
                                                          0.0005408759509716155,
                                                          0.017433817013255477,
                                                          0.0018366100135610477,
                                                          0.013578694121716223,
                                                          0.011286188143139364,
                                                          0.005320741349942404,
                                                          0.002924714763123512,
                                                          0.027303790029096588,
                                                          0.0...
            

In [66]:
rnd_search_cv.best_params_

{'n_neurons': 88, 'n_hidden': 4, 'learning_rate': 0.008836435280563442}

In [67]:
model = rnd_search_cv.best_estimator_.model
model

In [68]:
y_pred_dnn = model.predict(X_test)
mse_dnn = mean_squared_error(y_test, y_pred_dnn)
mse_dnn

0.0012217008133820835

In [72]:
# model.save_weights("./models")
model.save("./models/first_reg.h5")

In [42]:
X_train.shape

(2457600, 2)

In [71]:
# model= keras.models.Sequential()
# model.add(keras.layers.Dense(30, activation = "relu", input_shape = x_train.shape[1:]))
# for n_hidden in (100, 100, 50, 50, 50):
#     model.add(keras.layers.Dense(n_hidden, activation="selu", kernel_initializer="lecun_normal")),
# model.add(keras.layers.Dense(1))

In [72]:
model.compile(loss="mean_squared_error", optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999))

In [74]:
history = model.fit(X_train, y_train, epochs=20,
                      validation_data=(X_valid, y_cv))

Epoch 1/20
31144/76800 [===========>..................] - ETA: 32s - loss: 0.0635

KeyboardInterrupt: 

In [55]:
mse_test = model.evaluate(x_test, y_test)

25600/25600 [==============================] - 7s 276us/step - loss: 0.0636


## ML

In [11]:
param_grid = [
        {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
        {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True, verbose = 2)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END .....................max_features=2, n_estimators=3; total time=  14.5s
[CV] END .....................max_features=2, n_estimators=3; total time=  14.3s
[CV] END .....................max_features=2, n_estimators=3; total time=  14.2s
[CV] END .....................max_features=2, n_estimators=3; total time=  14.4s
[CV] END .....................max_features=2, n_estimators=3; total time=  14.5s
[CV] END ....................max_features=2, n_estimators=10; total time=  47.7s
[CV] END ....................max_features=2, n_estimators=10; total time=  47.9s


KeyboardInterrupt: 

In [73]:
forest_reg = RandomForestRegressor(max_features=2, n_estimators=30)
forest_reg.fit(X_train, y_train)
y_pred_forest = forest_reg.predict(X_test)

In [74]:
mse_forest = mean_squared_error(y_test, y_pred_forest)
mse_forest

0.001304256320709336

In [79]:
joblib.dump(forest_reg, "./models/RF_compressed.joblib", compress=3) 

['./models/RF_compressed.joblib']